# Признаки активности по времени суток

Импортируем библиотеки и делаем некоторые настройки

In [1]:
import pandas as pd
import numpy as np

In [2]:
from collections import Counter
import statistics

Загружаем данные за период с 15.06.2015г. по 19.05.2018г. о действиях, которые совершали студенты со стэпами и статусах сабмитов

In [3]:
events = pd.read_csv('DATA\\events_all_two_days.zip', compression='zip')

In [4]:
submissions = pd.read_csv('DATA\\submissions_all_two_days.zip', compression='zip')

In [5]:
#переименуем колонки
events = events.rename(columns={'action': 'action_submission_status'})
submissions = submissions.rename(columns={'submission_status': 'action_submission_status'})

In [6]:
#сливаем обе таблицы
data = pd.concat([events, submissions])

In [7]:
del events
del submissions

In [8]:
data.sort_values(by=['user_id', 'timestamp'], axis=0, inplace=True)

Преобразование даты из формата unix date (timestamp) в привычный формат

In [9]:
data['date'] = pd.to_datetime(data.timestamp, unit='s')

Создание отдельной колонки с часовыми отметками

In [10]:
data['hour'] = data.date.dt.hour

Разделим все действия по часам, а потом разделим на дневные и ночные:  
* дневные с 8:00 до 20:00 ч;
* ночные с 20:00 до 8:00 ч.

In [11]:
times_feature = data.groupby(['user_id', 'hour']).step_id.count() \
        .to_frame().reset_index().rename(columns={'step_id': 'steps_hour'})

In [12]:
times_feature.sort_values(by=['user_id', 'hour'], axis=0, inplace=True)

In [13]:
day = times_feature[(times_feature.hour >= 8) & (times_feature.hour < 20)]

In [14]:
day_feature = day.groupby(['user_id']).steps_hour.sum().to_frame() \
              .reset_index().rename(columns={'steps_hour': 'steps_day'})

In [15]:
night =times_feature[(times_feature.hour < 8) | (times_feature.hour >= 20)]

In [16]:
night_feature = night.groupby(['user_id']).steps_hour.sum().to_frame() \
            .reset_index().rename(columns={'steps_hour': 'steps_night'})

Соберем все в одну таблицу

In [17]:
day_feature = day_feature.merge(night_feature, on='user_id', how='outer')

In [18]:
day_feature.sort_values(by=['user_id'], axis=0, inplace=True)

In [19]:
day_feature = day_feature.fillna(0)

Еще признак, отношение

In [20]:
day_feature['day_ratio'] = day_feature.steps_day / \
                        (day_feature.steps_day + day_feature.steps_night)  

In [21]:
# проверка кол-ва пользователей
day_feature.shape

(25418, 4)

Сколько часов провели пользователи на курсе в течении 2-х дней, отношение количества часов к 48 часам

In [22]:
hours = data.groupby(['user_id', 'day']).hour.nunique().to_frame() \
            .reset_index().rename(columns={'hour': 'hours_day'})

In [23]:
del data

In [24]:
# суммирование по дням
hours = hours.groupby(['user_id']).hours_day.sum().to_frame() \
             .reset_index().rename(columns={'hours_day': 'hours_sum'})

In [25]:
# относительный признак
hours['hours_ratio'] = hours.hours_sum / hours.hours_sum.max()

In [26]:
# объединяем таблицы
day_feature = day_feature.merge(hours, on='user_id', how='inner')

Сохраним результаты в файл:

In [27]:
#day_feature.to_csv('DATA\\times_feature_events_all_NEW.csv', index=False)

In [28]:
day_feature

,user_id,steps_day,steps_night,day_ratio,hours_sum,hours_ratio
0,1,2.0,0.0,1.0,1,0.020408
1,2,31.0,0.0,1.0,1,0.020408
2,3,62.0,0.0,1.0,2,0.040816
3,4,3.0,0.0,1.0,1,0.020408
4,5,7.0,0.0,1.0,2,0.040816
...,...,...,...,...,...,...
25413,26796,29.0,0.0,1.0,2,0.040816
25414,26797,34.0,0.0,1.0,1,0.020408
25415,26798,0.0,42.0,0.0,2,0.040816
25416,26799,22.0,0.0,1.0,2,0.040816


In [29]:
day_feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25418 entries, 0 to 25417
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      25418 non-null  int64  
 1   steps_day    25418 non-null  float64
 2   steps_night  25418 non-null  float64
 3   day_ratio    25418 non-null  float64
 4   hours_sum    25418 non-null  int64  
 5   hours_ratio  25418 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 1.4 MB
